In [6]:
#!pip install wordcloud
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import FreqDist
import matplotlib.pyplot as plt
import nltk

import matplotlib.pyplot as plt
import numpy as np
import csv
import sklearn
from sklearn import datasets
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC

In [2]:
nlp = pd.read_csv('nlp.csv')

In [3]:
nlp

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,...,main_cat,also_buy,also_view,similar_item,price,date,details,tech1,tech2,fit
0,0,2.0,False,"11 24, 2007",A1TOKUIUSJ77DU,B000PH9NNS,Gideon I. Gartner,"For former users of touch screens, this machin...",Tough Going,1195862400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
1,1,4.0,False,"11 21, 2007",A2SK1UQ9HI9ZYS,B000PH9NNS,Ernesto D. Jimenez Perez,The product is fantastic but the software that...,Excellent phone Software in Beta phase,1195603200,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
2,2,5.0,True,"11 19, 2007",A1DQVXBQATAK2R,B000PH9NNS,Aldo Bender,I bought this phone to replace my Nokia 9300 a...,A real smartphone,1195430400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
3,3,5.0,False,"09 25, 2007",A1PTDL2WFBL6GV,B000PH9NNS,ZANE Farooq,"I've used Nokia phones all my life, except wel...",Nokia...Wow,1190678400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
4,4,4.0,False,"08 21, 2007",A3QP1VWPQS9S9D,B000PH9NNS,Paolo M. Reyes,The Nokia E90 Communicator is a mobile office ...,"E90, Eager to transform the way people experie...",1187654400,...,Cell Phones & Accessories,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10058,10058,1.0,True,"08 22, 2016",A3IESEK173F6ZZ,B01E6O625W,Kathleen Mulkey,"They are horrible, don't receive or make calls...",S4 sucks,1471824000,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN
10059,10059,5.0,True,"08 12, 2016",A1JCA9NI8WSAEL,B01E6O625W,dasun kloss,Awesome phone!! Works great! Delivered early!!...,Five Stars,1470960000,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN
10060,10060,5.0,True,"08 4, 2016",A3MYU5E9XUQJSO,B01E6O625W,Gail Roe,the phone work great got it before it was supp...,Five Stars,1470268800,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN
10061,10061,5.0,True,"07 19, 2016",ANWZUOXV9MDGS,B01E6O625W,Kenneth Darnell,Great phone and a good deal,Five Stars,1468886400,...,Cell Phones & Accessories,NaN,NaN,NaN,399.99,NaN,NaN,NaN,NaN,NaN


In [3]:
nlp['target'] = np.where(nlp.overall >3, 1,
                       np.where(nlp.overall <3.0, 0,
                       'unkown'))

In [4]:
nlp.drop(nlp[nlp['target'] =='unkown'].index, axis=0, inplace=True)
nlp.reset_index(drop=True, inplace=True)
nlp['target'] = pd.to_numeric(nlp['target'])
nlp.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,...,also_buy,also_view,similar_item,price,date,details,tech1,tech2,fit,target
0,0,2.0,False,"11 24, 2007",A1TOKUIUSJ77DU,B000PH9NNS,Gideon I. Gartner,"For former users of touch screens, this machin...",Tough Going,1195862400,...,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,0
1,1,4.0,False,"11 21, 2007",A2SK1UQ9HI9ZYS,B000PH9NNS,Ernesto D. Jimenez Perez,The product is fantastic but the software that...,Excellent phone Software in Beta phase,1195603200,...,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,1
2,2,5.0,True,"11 19, 2007",A1DQVXBQATAK2R,B000PH9NNS,Aldo Bender,I bought this phone to replace my Nokia 9300 a...,A real smartphone,1195430400,...,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,1
3,3,5.0,False,"09 25, 2007",A1PTDL2WFBL6GV,B000PH9NNS,ZANE Farooq,"I've used Nokia phones all my life, except wel...",Nokia...Wow,1190678400,...,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,1
4,4,4.0,False,"08 21, 2007",A3QP1VWPQS9S9D,B000PH9NNS,Paolo M. Reyes,The Nokia E90 Communicator is a mobile office ...,"E90, Eager to transform the way people experie...",1187654400,...,NaN,"['B075FL4H89', 'B000PEOLAG', 'B00851Q9L0', 'B0...","[{'asin': '', 'href': '', 'name': 'Nokia E90 C...",404.47,NaN,NaN,NaN,NaN,NaN,1


In [7]:
nlp.reviewText.dropna(inplace = True)

In [8]:
nlp.reviewText = nlp.reviewText.str.lower()
nlp['clean_up_review']= nlp.reviewText.str.replace('\n',''). \
                                          str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')
nlp.reset_index(drop=True, inplace=True)

In [10]:
nlp_train, nlp_test = train_test_split(nlp[['clean_up_review','target']],test_size=0.3)

In [114]:
import eli5
from sklearn.feature_extraction import text

In [146]:
## test the optimal parameters
C_range = np.arange(1,11,1)
max_test_f1 = 0
for i in C_range:
    for j in range(1, 10, 1):
        lr_model = LogisticRegression(C=i)
        my_stop_words = text.ENGLISH_STOP_WORDS.union(["love",'great','best','like','better','phone'])
        vec = CountVectorizer(ngram_range=(3,3),min_df = j, stop_words = my_stop_words)
        pipe = make_pipeline(vec, lr_model)
        pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
        train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
        train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
        test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
        test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
        train_f1, test_f1
        print('train_f1:',train_f1,'test_f1:', test_f1, ",","C", i, "min_df",  j)
        if( test_f1 > max_test_f1):
            opt_C = i
            opt_min_df = j 
            max_test_f1 = test_f1

train_f1: 0.953297952994693 test_f1: 0.7368234877962504 , C 1 min_df 1
train_f1: 0.7865049279757392 test_f1: 0.7435443933498408 , C 1 min_df 2
train_f1: 0.7652767247915088 test_f1: 0.7438981252210826 , C 1 min_df 3
train_f1: 0.7501137225170583 test_f1: 0.7403608065086664 , C 1 min_df 4
train_f1: 0.7451099317664898 test_f1: 0.7368234877962504 , C 1 min_df 5
train_f1: 0.7426838514025776 test_f1: 0.7357622921825256 , C 1 min_df 6
train_f1: 0.7423805913570887 test_f1: 0.7357622921825256 , C 1 min_df 7
train_f1: 0.7402577710386656 test_f1: 0.733639900955076 , C 1 min_df 8
train_f1: 0.7378316906747536 test_f1: 0.7318712415988682 , C 1 min_df 9
train_f1: 0.9742228961334344 test_f1: 0.7371772196674921 , C 2 min_df 1
train_f1: 0.8141015921152389 test_f1: 0.7488503714184648 , C 2 min_df 2
train_f1: 0.7702805155420773 test_f1: 0.744251857092324 , C 2 min_df 3
train_f1: 0.7514783927217589 test_f1: 0.7403608065086664 , C 2 min_df 4
train_f1: 0.7451099317664898 test_f1: 0.7361160240537673 , C 2 min_

C:\Users\xiaos\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.8250189537528431 test_f1: 0.7509727626459144 , C 3 min_df 2
train_f1: 0.7725549658832449 test_f1: 0.7460205164485321 , C 3 min_df 3
train_f1: 0.7520849128127369 test_f1: 0.7403608065086664 , C 3 min_df 4
train_f1: 0.7460197119029568 test_f1: 0.7364697559250089 , C 3 min_df 5
train_f1: 0.7432903714935557 test_f1: 0.7357622921825256 , C 3 min_df 6
train_f1: 0.7428354814253222 test_f1: 0.7350548284400424 , C 3 min_df 7
train_f1: 0.7401061410159212 test_f1: 0.7339936328263177 , C 3 min_df 8
train_f1: 0.7379833206974981 test_f1: 0.7322249734701097 , C 3 min_df 9
train_f1: 0.9746777862016679 test_f1: 0.7385921471524585 , C 4 min_df 1
train_f1: 0.8295678544351783 test_f1: 0.7523876901308808 , C 4 min_df 2
train_f1: 0.7733131159969674 test_f1: 0.7456667845772903 , C 4 min_df 3
train_f1: 0.7523881728582259 test_f1: 0.740007074637425 , C 4 min_df 4
train_f1: 0.7463229719484459 test_f1: 0.7357622921825256 , C 4 min_df 5
train_f1: 0.7434420015163001 test_f1: 0.735408560311284 , C 4 min

C:\Users\xiaos\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.7525398028809704 test_f1: 0.740007074637425 , C 5 min_df 4
train_f1: 0.7466262319939347 test_f1: 0.7357622921825256 , C 5 min_df 5
train_f1: 0.7434420015163001 test_f1: 0.735408560311284 , C 5 min_df 6
train_f1: 0.7428354814253222 test_f1: 0.735408560311284 , C 5 min_df 7
train_f1: 0.7401061410159212 test_f1: 0.7339936328263177 , C 5 min_df 8
train_f1: 0.7378316906747536 test_f1: 0.7322249734701097 , C 5 min_df 9
train_f1: 0.9849886277482942 test_f1: 0.7392996108949417 , C 6 min_df 1
train_f1: 0.8333586050037908 test_f1: 0.7527414220021224 , C 6 min_df 2


...

train_f1: 0.8350265352539803 test_f1: 0.7513264945171559 , C 10 min_df 2


C:\Users\xiaos\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.7757391963608794 test_f1: 0.7453130527060489 , C 10 min_df 3


C:\Users\xiaos\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


train_f1: 0.7534495830174375 test_f1: 0.7396533427661831 , C 10 min_df 4
train_f1: 0.7466262319939347 test_f1: 0.7350548284400424 , C 10 min_df 5
train_f1: 0.7434420015163001 test_f1: 0.7347010965688009 , C 10 min_df 6
train_f1: 0.7431387414708113 test_f1: 0.735408560311284 , C 10 min_df 7
train_f1: 0.7404094010614102 test_f1: 0.7329324372125927 , C 10 min_df 8
train_f1: 0.7379833206974981 test_f1: 0.7322249734701097 , C 10 min_df 9


In [147]:
print(opt_C,
opt_min_df,
max_test_f1)

6 2 0.7527414220021224


In [148]:
## fit in the lr model
lr_model = LogisticRegression(C=6)
my_stop_words = text.ENGLISH_STOP_WORDS.union(["love",'great','best','like','better','phone'])
vec = CountVectorizer(ngram_range = (3,3), min_df= 2,stop_words = my_stop_words)
pipe = make_pipeline(vec, lr_model)
pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
train_f1, test_f1  

(0.8333586050037908, 0.7527414220021224)

In [151]:
# use eli5 to show the features
eli5.show_weights(pipe, top=(20,20))

Weight?,Feature
+2.128,high quality pics
+2.034,brand new box
+2.031,just make sure
+1.997,battery lasts day
+1.942,battery left plugged
+1.934,battery life significantly
+1.911,using usb cord
+1.892,long battery life
+1.891,fast easy use
+1.879,reason gave stars


In [51]:
## same with rf model
rf_model = RandomForestClassifier(n_estimators = 300, random_state = 50)
my_stop_words = text.ENGLISH_STOP_WORDS.union(["love",'great','best','like','better','phone'])
vec = TfidfVectorizer(ngram_range = (3,3), stop_words = my_stop_words,sublinear_tf = True)
pipe = make_pipeline(vec, rf_model)
pipe.fit(nlp_train.clean_up_review.values.astype('U'), nlp_train.target)
train_preds = pipe.predict(nlp_train.clean_up_review.values.astype('U'))
train_f1 = f1_score(nlp_train.target, train_preds, average='micro')
test_preds = pipe.predict(nlp_test.clean_up_review.values.astype('U'))
test_f1 = f1_score(nlp_test.target, test_preds, average='micro')
train_f1, test_f1  

(0.9834723275208491, 0.7417757339936328)

In [49]:
eli5.show_weights(pipe, top=100)

Weight,Feature
0.0023 ± 0.0027,stopped working months
0.0019 ± 0.0030,bought new battery
0.0018 ± 0.0030,waste time money
0.0012 ± 0.0025,screen stopped working
0.0010 ± 0.0017,stop working months
0.0010 ± 0.0020,doesn work properly
0.0010 ± 0.0023,read sim card
0.0009 ± 0.0028,galaxy s7 edge
0.0009 ± 0.0016,didnt work properly
0.0009 ± 0.0015,discribe lot dents
